In [1]:
import yaml
import time
from datetime import datetime
from megadetector.detection import run_detector
import pandas as pd

import sys
sys.path.append('../scripts')  # Adjust the path as needed to point to the directory containing utils.py

from alert_system_utils import detector, classifier, set_device, check_emails, extract_and_update_camera_info, update_camera_data_dataframe, batch_classification, current_time, generate_alert_caption, check_counts_and_species

In [2]:
# Load settings from configuration file
with open('../config.yaml') as file:
    config = yaml.safe_load(file)

IMAP_HOST = config['imap_config']['host']
EMAIL_USER = config['imap_config']['user']
EMAIL_PASS = config['imap_config']['password']
TELEGRAM_BOT_TOKEN = config['telegram_config']['bot_token']
TELEGRAM_CHAT_ID = '-1002249589791' # replace with config after tests

# Detection and Classification Model Settings
DETECTOR_MODEL_PATH = '../models/md_v5a.0.0.pt'
DETECTOR_CLASSES = ["animal", "human", "vehicle"]
DETECTION_THRESHOLD = 0.10

BACKBONE = 'vit_large_patch14_dinov2'
CLASSIFIER_MODEL_PATH = '../models/deepfaune-vit_large_patch14_dinov2.lvd142m.pt'
CLASSIFIER_CLASSES = ["Badger", "Ibex", "Red Deer", "Chamois", "Cat", "Goat", "Roe Deer", "Dog", "Squirrel", "Equid", "Genet", "Hedgehog", "Lagomorph", "Wolf", "Lynx", "Marmot", "Micromammal", "Mouflon", "Sheep", "Mustelid", "Bird", "Bear", "Nutria", "Fox", "Wild Boar", "Cow"]
SPECIES_OF_INTEREST = ["Wild Boar", "Bear"]
CLASSIFICATION_THRESHOLD = 0.20

CAPTURE_DATABASE_PATH = '../data/capture_database.csv'
CAMERA_LOCATIONS_PATH = '../data/camera_locations.csv'
ALERT_LANGUAGE = "en" # Enter 'en' for English or 'ro' for Romanian

# Initialise the Detection and Classifier Models
device = set_device()
detector_model = run_detector.load_detector(DETECTOR_MODEL_PATH)
print("Loading classifier...")
start_time = time.time()
classifier_model = classifier(CLASSIFIER_MODEL_PATH, BACKBONE, CLASSIFIER_CLASSES, device)
end_time = time.time()
print(f"Loaded classifier in {(end_time - start_time):.2f} seconds")

Imported YOLOv5 as utils.*
Using PyTorch version 2.3.0+cu121


Fusing layers... 
Fusing layers... 
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs


Sending model to GPU
Loaded model in 1.44 seconds
Loading classifier...
Loaded classifier in 4.01 seconds


In [9]:
if __name__ == "__main__":
    print(f"\n{current_time()} | Monitoring {EMAIL_USER} for new messages...")
    while True:
        try:
            images, camera_id, temp_deg_c, img_date, img_time, battery, sd_memory = \
                check_emails(IMAP_HOST, EMAIL_USER, EMAIL_PASS)
            
            if camera_id:

                camera_make, gps, location, map_url, battery, sd_memory = extract_and_update_camera_info(CAMERA_LOCATIONS_PATH, camera_id, battery, sd_memory)
                
                # TO DO: if camera_id is UOVision, wait 10, check for another, then append or continue.
                # check subject to determine if uovision, then only open if it is

                df = pd.read_csv(CAPTURE_DATABASE_PATH)
                df = update_camera_data_dataframe(df, len(images), camera_id, camera_make, img_date, img_time, temp_deg_c, battery, sd_memory, location, gps, map_url)

                if images:

                    df = detector(df, detector_model, images, DETECTION_THRESHOLD)
                    
                    df = batch_classification(df, classifier_model, images, CLASSIFICATION_THRESHOLD) # test classification threshold

                    if check_counts_and_species(df, images):

                        df, alert_caption, human_warning, priority_alert = generate_alert_caption(df, len(images), SPECIES_OF_INTEREST, EMAIL_USER, ALERT_LANGUAGE)

                        alert_images = annotate_images(df, images)

                        send_alert_to_telegram(TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID, alert_images, alert_caption)

                        # print(f"\n\n Test Alert Caption: \n\n {alert_caption}")

                    # df = save_images(df, images)
                
                df.to_csv(CAPTURE_DATABASE_PATH, index=False)
                print(f"{current_time()} | Capture Database Updated: {CAPTURE_DATABASE_PATH}")
                #del df
                
                # delete email

            else:
                time.sleep(1)

        except KeyboardInterrupt:
            print(f"{current_time()} | Interrupted by user")
            break

        # except Exception as e:
        #    print(f"An error occurred: {e}")
        #    print(f"\nMonitoring {EMAIL_USER} for new messages...")
        #    continue



24-06-02 00:02:33 | Monitoring fcccameratraps@gmail.com for new messages...

24-06-02 00:02:41 | Email Received.
24-06-02 00:02:41 | Images: 5, Camera ID: WILDBOAR01, Camera Make: Wilsus
24-06-02 00:02:41 | Date: 2024-05-30, Time: 04:59:16, Temperature: None
24-06-02 00:02:41 | Battery: 80%, SD Memory: 99%
24-06-02 00:02:41 | Location: Albesti, GPS: X: 25.00330022, Y: 45.32415226, Z: 813m, Map URL: https://maps.app.goo.gl/DueDZiWQGinkELuU9
24-06-02 00:02:42 | Image 1: Animal Count = 1, Human Count = 0, Vehicle Count = 0
24-06-02 00:02:42 | Image 2: Animal Count = 1, Human Count = 0, Vehicle Count = 0
24-06-02 00:02:42 | Image 3: Animal Count = 1, Human Count = 0, Vehicle Count = 0
24-06-02 00:02:42 | Image 4: Animal Count = 2, Human Count = 0, Vehicle Count = 0
24-06-02 00:02:43 | Image 5: Animal Count = 4, Human Count = 0, Vehicle Count = 0
24-06-02 00:02:43 | Image 1, Detection 1 (90.90% confidence), Species: Wild Boar (99.78% confidence)
24-06-02 00:02:43 | Image 2, Detection 1 (89

In [7]:
from PIL import Image, ImageDraw
import io
from PIL import Image, ImageDraw
import io

def annotate_images(df, images):
    image_list = []
    
    # Extract the last len(images) rows
    last_rows = df.iloc[-len(images):]

    for image, row in zip(images, last_rows.iterrows()):
        index, row_data = row
        draw = ImageDraw.Draw(image)

        detection_boxes = row_data['Detection Boxes']
        detection_classes = row_data['Detection Classes']
        detection_confidences = row_data['Detection Confidences']
        species_classes = row_data['Species Classes']
        species_confidences = row_data['Species Confidences']
        
        species_idx = 0  # to keep track of the current species index

        for box, d_class, d_conf in zip(detection_boxes, detection_classes, detection_confidences):
            # Convert relative coordinates to absolute coordinates
            width, height = image.size
            x1 = int(box[0] * width)
            y1 = int(box[1] * height)
            x2 = int((box[0] + box[2]) * width)
            y2 = int((box[1] + box[3]) * height)
            
            # Set color and label based on detection class
            if d_class == '1':  # Animal
                color = 'yellow'
                label = f"{species_classes[species_idx]} {species_confidences[species_idx] * 100:.0f}%"
                species_idx += 1
            elif d_class == '2':  # Human
                color = 'blue'
                label = f"Human {d_conf * 100:.0f}%"
            elif d_class == '3':  # Vehicle
                color = 'red'
                label = f"Vehicle {d_conf * 100:.0f}%"
            else:
                color = 'green'
                label = f"Unknown {d_conf * 100:.0f}%"
                
            # Draw bounding box
            draw.rectangle([x1, y1, x2, y2], outline=color, width=3)
            draw.text((x1, y1 - 10), label, fill=color)
        
        image_list.append(image)
    
    return image_list


In [8]:
import requests
import io
import json
import time

def send_alert_to_telegram(bot_token, chat_id, photos, caption):
    url = f"https://api.telegram.org/bot{bot_token}/sendMediaGroup"
    media = []
    files = {}

    for idx, photo in enumerate(photos):
        buf = io.BytesIO()
        photo.save(buf, format='JPEG')
        buf.seek(0)
        file_name = f'photo{idx}.jpg'
        files[file_name] = buf.getvalue()  # Store the bytes data

        media.append({
            'type': 'photo',
            'media': f'attach://{file_name}',
            'caption': caption if idx == 0 else ""  # Caption only for the first image
        })

    # Prepare the data and files for the request
    data = {
        'chat_id': chat_id, 
        'media': json.dumps(media),
        'parse_mode': 'Markdown'  # Enable Markdown parsing
    }
    files = {name: (name, io.BytesIO(bytes_data), 'image/jpeg') for name, bytes_data in files.items()}

    response = requests.post(url, data=data, files=files)
    if response.status_code == 429:
        # Handle rate limit
        retry_after = int(response.headers.get("Retry-After", 1))
        print(f"Rate limited. Retrying after {retry_after} seconds.")
        time.sleep(retry_after)
        response = requests.post(url, data=data, files=files)
    
    response.raise_for_status()
    print("Alert sent.")

# Example usage
# send_alert_to_telegram(TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID, alert_images, "*:rotating_light: ALERT:rotating_light:*")
